In [288]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pymysql
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
%matplotlib inline
conn = pymysql.connect(host='localhost', user='root', password='Sk1ttles', db='nba_stats_prod')

In [289]:
raw_sql = """
select adv.win_lose,
       bm.home_away,
       adv.game_length,
       adv.ast_pct,
       adv.ast_to_to,
       adv.ast_ratio,
       adv.offensive_reb_pct,
       adv.defensive_reb_pct,
       adv.reb_pct,
       adv.to_pct,
       adv.effective_fg_pct,
       adv.ts_pct,
       adv.pace,
       fig.fta_rate,
       fig.tov_pct,
       fig.oreb_pct,
       fig.opp_effective_fg_pct,
       fig.opp_fta_rate,
       fig.opp_tov_pct,
       fig.opp_off_reb_pct,
       misc.points_off_to,
       misc.second_chance_pts,
       misc.fbps,
       misc.pts_in_paint,
       misc.opp_pts_off_to,
       misc.opp_second_chance_pts,
       misc.opp_fbps,
       misc.opp_pts_in_paint,
       score.pct_2pt_fg,
       score.pct_3pt_fg,
       score.pct_pts_2pt,
       score.pct_pts_2pt_mr,
       score.pct_pts_3pt,
       score.pct_pts_fbps,
       score.pct_pts_ft,
       score.pct_pts_off_to,
       score.pct_pts_pitp,
       score.2pt_fgm_ast_pct,
       score.3pt_fgm_ast_pct,
       score.3pt_fgm_uast_pct,
       score.fgm_pct_ast,
       score.fgm_pct_uast,
       trad.fga,
       trad.fg_pct,
       trad.3pa,
       trad.3p_pct,
       trad.fta,
       trad.ft_pct,
       trad.oreb,
       trad.dreb,
       trad.tot_reb,
       trad.ast,
       trad.stl,
       trad.blk,
       trad.personal_fouls
from(

    select b.team, b.game_hash, b.home_away
    from box_scores_map_view as b
    inner join game_date_lookup as lu on b.game_date = lu.day
    where lu.season > 1995 and
          b.game_hash not in (

              select game_hash
              from box_scores_map_view
              where team like 'Washington%' and
                    game_date > (select max(game_date) -26  from box_scores_map_view where team like 'Washington%')

              )

        )  as bm
inner join team_advanced_boxscore_stats as adv on ( (bm.team = adv.team) and (bm.game_hash = adv.game_hash) )
inner join team_figure4_boxscore_stats as fig on ( (adv.team = fig.team) and (adv.game_hash = fig.game_hash) )
inner join team_misc_boxscore_stats as misc on ( (fig.team = misc.team) and (fig.game_hash = misc.game_hash) )
inner join team_scoring_boxscore_stats as score on ( (misc.team = score.team) and (misc.game_hash = score.game_hash) )
inner join team_traditional_boxscore_stats as trad on ( (score.team = trad.team) and (score.game_hash = trad.game_hash) )
"""

In [290]:
team_stats = pd.read_sql(raw_sql, con=conn)
team_stats.columns

Index(['win_lose', 'home_away', 'game_length', 'ast_pct', 'ast_to_to',
       'ast_ratio', 'offensive_reb_pct', 'defensive_reb_pct', 'reb_pct',
       'to_pct', 'effective_fg_pct', 'ts_pct', 'pace', 'fta_rate', 'tov_pct',
       'oreb_pct', 'opp_effective_fg_pct', 'opp_fta_rate', 'opp_tov_pct',
       'opp_off_reb_pct', 'points_off_to', 'second_chance_pts', 'fbps',
       'pts_in_paint', 'opp_pts_off_to', 'opp_second_chance_pts', 'opp_fbps',
       'opp_pts_in_paint', 'pct_2pt_fg', 'pct_3pt_fg', 'pct_pts_2pt',
       'pct_pts_2pt_mr', 'pct_pts_3pt', 'pct_pts_fbps', 'pct_pts_ft',
       'pct_pts_off_to', 'pct_pts_pitp', '2pt_fgm_ast_pct', '3pt_fgm_ast_pct',
       '3pt_fgm_uast_pct', 'fgm_pct_ast', 'fgm_pct_uast', 'fga', 'fg_pct',
       '3pa', '3p_pct', 'fta', 'ft_pct', 'oreb', 'dreb', 'tot_reb', 'ast',
       'stl', 'blk', 'personal_fouls'],
      dtype='object')

In [263]:
team_stats.groupby('home_away')['pts'].plot.hist(bins=50)

KeyError: 'Column not found: pts'

In [264]:
#team_stats['away_score'].plot.hist(bins=30)

In [265]:
#def create_dumb_dumb(home):
 #   print(home)
   # if scores[0] < scores[1]:
    #    print('L')
        #return 'L'
   # elif scores[0] > scores[1]:
    #    print('w')
        #return 'W'

In [266]:
#home_wins = team_stats[team_stats['home_score'] > team_stats['away_score']]

In [267]:
#home_wins['result'] = 'W'

In [268]:
#home_wins.head()

In [269]:
#home_loses = team_stats[team_stats['away_score'] > team_stats['home_score']]

In [270]:
#home_loses['result'] = 'L'

In [271]:
#team_stats = pd.concat([home_wins, home_loses])

In [291]:
result = pd.get_dummies(team_stats['win_lose'], drop_first=True)

In [292]:
result.head()

,W
0,0
1,1
2,0
3,1
4,1


In [293]:
team_stats = pd.concat([team_stats, result], axis=1)

In [294]:
rest_home = pd.get_dummies(team_stats['home_away'], drop_first=True)

In [295]:
team_stats = pd.concat([team_stats, rest_home], axis=1)

In [296]:
team_stats.drop(['home_away', 'win_lose'], axis=1, inplace=True)

In [297]:
team_stats.columns

Index(['game_length', 'ast_pct', 'ast_to_to', 'ast_ratio', 'offensive_reb_pct',
       'defensive_reb_pct', 'reb_pct', 'to_pct', 'effective_fg_pct', 'ts_pct',
       'pace', 'fta_rate', 'tov_pct', 'oreb_pct', 'opp_effective_fg_pct',
       'opp_fta_rate', 'opp_tov_pct', 'opp_off_reb_pct', 'points_off_to',
       'second_chance_pts', 'fbps', 'pts_in_paint', 'opp_pts_off_to',
       'opp_second_chance_pts', 'opp_fbps', 'opp_pts_in_paint', 'pct_2pt_fg',
       'pct_3pt_fg', 'pct_pts_2pt', 'pct_pts_2pt_mr', 'pct_pts_3pt',
       'pct_pts_fbps', 'pct_pts_ft', 'pct_pts_off_to', 'pct_pts_pitp',
       '2pt_fgm_ast_pct', '3pt_fgm_ast_pct', '3pt_fgm_uast_pct', 'fgm_pct_ast',
       'fgm_pct_uast', 'fga', 'fg_pct', '3pa', '3p_pct', 'fta', 'ft_pct',
       'oreb', 'dreb', 'tot_reb', 'ast', 'stl', 'blk', 'personal_fouls', 'W',
       'Home'],
      dtype='object')

In [298]:
X = team_stats.drop('W', axis=1)
y = team_stats['W']

In [299]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.33)

In [300]:
lg = LogisticRegression()

In [301]:
lg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [302]:
predictions = lg.predict(X_test)

In [303]:
print(classification_report(y_test, predictions))

             precision    recall  f1-score   support

          0       0.97      0.97      0.97      7990
          1       0.97      0.97      0.97      7900

avg / total       0.97      0.97      0.97     15890



In [304]:
print(confusion_matrix(y_test, predictions))

[[7737  253]
 [ 261 7639]]


In [305]:
pd.DataFrame(lg.coef_[0],X_test.columns,columns=['Coefficient'])

,Coefficient
game_length,-2.130044
ast_pct,0.349291
ast_to_to,0.118738
ast_ratio,-0.205962
offensive_reb_pct,0.162868
defensive_reb_pct,0.333960
reb_pct,-0.017772
to_pct,0.138769
effective_fg_pct,-0.140731
ts_pct,1.688807


In [306]:
lg.intercept_.astype(float)

array([ 0.00086762])